# Build a Contextual Retrieval based RAG System

In [ ]:
%run setup.ipynb

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [1]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Loading and Processing the Data

### Load and Process JSON Documents

In [2]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(file_path='../../docs/wikidata_rag_demo.jsonl',
                    jq_schema='.',
                    text_content=False,
                    json_lines=True)
wiki_docs = loader.load()

In [3]:
import json
from langchain.docstore.document import Document
wiki_docs_processed = []

for doc in wiki_docs:
    doc = json.loads(doc.page_content)
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
        "source": "Wikipedia",
        "page": 1
    }
    data = ' '.join(doc['paragraphs'])
    wiki_docs_processed.append(Document(page_content=data, metadata=metadata))

### Load and Process PDF documents

#### Create Chunk Contexts for Contextual Retrieval

![](https://i.imgur.com/LRhKHzk.png)

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [5]:
# create chunk context generation chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


def generate_chunk_context(document, chunk):

    chunk_process_prompt = """You are an AI assistant specializing in research paper analysis.
                            Your task is to provide brief, relevant context for a chunk of text
                            based on the following research paper.

                            Here is the research paper:
                            <paper>
                            {paper}
                            </paper>

                            Here is the chunk we want to situate within the whole document:
                            <chunk>
                            {chunk}
                            </chunk>

                            Provide a concise context (3-4 sentences max) for this chunk,
                            considering the following guidelines:

                            - Give a short succinct context to situate this chunk within the overall document
                            for the purposes of improving search retrieval of the chunk.
                            - Answer only with the succinct context and nothing else.
                            - Context should be mentioned like 'Focuses on ....'
                            do not mention 'this chunk or section focuses on...'

                            Context:
                        """

    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)

    agentic_chunk_chain = (prompt_template
                                |
                            chatgpt
                                |
                            StrOutputParser())

    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})

    return context

In [6]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):

    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)

    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])
    contextual_chunks = []
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),
            'page': chunk_metadata['page'],
            'source': chunk_metadata['source'],
            'title': chunk_metadata['source'].split('/')[-1]
        }
        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [9]:
from glob import glob

pdf_files = glob('../../docs/*.pdf')
pdf_files

['../../docs/layoutparser_paper.pdf',
 '../../docs/cnn_paper.pdf',
 '../../docs/vision_transformer.pdf',
 '../../docs/resnet_paper.pdf',
 '../../docs/WEB_How_and_Why_to_UseLLMs_for_Chunk_Based_Information_Retrieval_Carlo_Peron_Oct_2024_TowardsDataScience.pdf',
 '../../docs/attention_paper.pdf',
 '../../docs/Vision Transformers.pdf']

Only loading data from specific pdf's to handle GPT4 out of context error

In [10]:
paper_docs = []
for fp in pdf_files:
    if ('attention' in fp) or ('transformer' in fp) or ('vision' in fp):
        paper_docs.extend(create_contextual_chunks(fp))

Loading pages: ../../docs/vision_transformer.pdf
Chunking pages: ../../docs/vision_transformer.pdf
Generating contextual chunks: ../../docs/vision_transformer.pdf
Finished processing: ../../docs/vision_transformer.pdf

Loading pages: ../../docs/attention_paper.pdf
Chunking pages: ../../docs/attention_paper.pdf
Generating contextual chunks: ../../docs/attention_paper.pdf
Finished processing: ../../docs/attention_paper.pdf



In [11]:
len(paper_docs)

44

### Combine all document chunks in one list

In [13]:
len(wiki_docs_processed)

1801

In [14]:
total_docs = wiki_docs_processed + paper_docs
len(total_docs)

1845

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [15]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=total_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [16]:
# load from disk
chroma_db = Chroma(persist_directory="./my_context_db",
                   collection_name='my_context_db',
                   embedding_function=openai_embed_model)

In [17]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 5 similar documents based on the user input query

In [18]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

In [19]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

In [20]:
query = "what is machine learning?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': '564928', 'page': 1, 'source': 'Wikipedia', 'title': 'Machine learning'}
Content Brief:


Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Metadata: {'id': '359370', 'page': 1, 'source': 'Wikipedia', 'title': 'Supervised learning'}
Content Brief:


In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the system simply learns how to get to these results correctly. Usually, such systems work with vectors. They get the training data and the result of the training as two vectors and produce a "classifier". Usually, the system uses inductive reasoning to generalize the training data.


Metadata: {'id': '663523', 'page': 1, 'source': 'Wikipedia', 'title': 'Deep learning'}
Content Brief:


Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, whic


Metadata: {'id': '6360', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial intelligence'}
Content Brief:


Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental facu


Metadata: {'id': '44742', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial neural network'}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.

In [21]:
query = "what is the difference between transformers and vision transformers?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': '5b62f899-a41f-4452-8c51-82407866b9b5', 'page': 7, 'source': '../../docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on a controlled scaling study of various models, including Vision Transformers and ResNets, to evaluate their transfer performance from the JFT-300M dataset. It highlights the performance versus pre-training cost, revealing that Vision Transformers generally outperform ResNets in terms of efficiency and scalability. Additionally, it discusses the implications of hybrid models and the potential for further scaling of Vision Transformers.
Published as a conference paper at ICLR 2021
4.4
SCALING STUDY
We perform a controlled scaling study of different models by evaluating transfer performance from
JFT-300M. In this setting data size does not bottleneck the models’ performances, and we assess
performance versus pre-training cost of each model. The model set includes: 7 ResNets, R50x1,
R50x2 R101x1, R152x1, R152x2, pre-trained for 7 epochs, plus R152x2 and R200x3 pre-trained
for 14 epochs; 6 Vision Transformers, ViT-B/32, B/16, L/32, L/16, pre-trained for 7 epochs, plus
L/16 and H/1


Metadata: {'id': 'f67497d2-411f-465b-84c4-c686183e95fd', 'page': 0, 'source': '../../docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the introduction of the Vision Transformer (ViT) model, which applies a standard Transformer architecture directly to image classification tasks by treating image patches as tokens. It highlights the limitations of traditional convolutional neural networks (CNNs) in computer vision and presents evidence that ViT can achieve competitive performance on various benchmarks with fewer computational resources when pre-trained on large datasets.
Published as a conference paper at ICLR 2021
AN IMAGE IS WORTH 16X16 WORDS:
TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE
Alexey Dosovitskiy∗,†, Lucas Beyer∗, Alexander Kolesnikov∗, Dirk Weissenborn∗,
Xiaohua Zhai∗, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer,
Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, Neil Houlsby∗,†
∗equal technical contribution, †equal advising
Google Research, Brain Team
{adosovitskiy, neilhoulsby}@google.com
ABSTRACT
While the Transformer architecture has become the de-facto standard for natural
language 


Metadata: {'id': '241f4ed4-9dd4-4cf1-b83f-bd8348556adf', 'page': 2, 'source': '../../docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding them, and utilizing a standard Transformer encoder for image classification tasks. It describes the model's design principles, including the use of position embeddings and the integration of a classification token, while referencing foundational work in Transformer architecture.
Published as a conference paper at ICLR 2021
Transformer Encoder
MLP 
Head
Vision Transformer (ViT)
*
Linear Projection of Flattened Patches
* Extra learnable
     [ cl ass]  embedding
1
2
3
4
5
6
7
8
9
0
Patch + Position 
Embedding
Class
Bird
Ball
Car
...
Embedded 
Patches
Multi-Head 
Attention
Norm
MLP
Norm
+
L x
+
Transformer Encoder
Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them,
add position embeddings, and feed the resulting sequence of vectors to a standard Transformer
encoder. In order to perform classiﬁ


Metadata: {'id': '817bc951-8051-48be-b7ff-fe40c55d58a3', 'page': 7, 'source': '../../docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the behavior of attention mechanisms in Vision Transformers, highlighting how attention distances vary across layers and the implications for model performance. It discusses the relationship between attention distance and network depth, as well as the role of hybrid models that incorporate convolutional layers. Additionally, it sets the stage for the subsequent exploration of self-supervised learning techniques in Transformers.
have consistently small attention distances in the low layers. This highly localized attention is
less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right),
suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the
attention distance increases with network depth. Globally, we ﬁnd that the model attends to image
regions that are semantically relevant for classiﬁcation (Figure 6).
4.6
SELF-SUPERVISION
Transformers show impressive performance on NLP tasks. However, much of thei


Metadata: {'id': '778c3e13-86fe-4aaf-811d-ecb6fbf01a7a', 'page': 1, 'source': '../../docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the performance of the Vision Transformer (ViT) when trained on large datasets, highlighting its ability to achieve state-of-the-art results in image recognition tasks despite lacking some inductive biases inherent to convolutional neural networks (CNNs). It also discusses related work in the field, particularly the application of self-attention mechanisms in image processing and comparisons with existing models.
Published as a conference paper at ICLR 2021
inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well
when trained on insufﬁcient amounts of data.
However, the picture changes if the models are trained on larger datasets (14M-300M images). We
ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent
results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When
pre-trained on the public ImageNet-21k dataset or the in-house JFT-300M dataset, ViT ap

## Build the RAG Pipeline

In [22]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [23]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    rag_prompt_template
      |
    chatgpt
)

In [24]:
from IPython.display import display, Markdown

query = "What is machine learning?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept was introduced by Arthur Samuel in 1959 and is rooted in artificial intelligence. Machine learning focuses on the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. These algorithms follow programmed instructions but can also adapt and improve their performance by building models from sample inputs.

Machine learning is particularly useful in scenarios where designing and programming explicit algorithms is impractical. Some common applications of machine learning include:

- Spam filtering
- Detection of network intruders or malicious insiders
- Optical character recognition (OCR)
- Search engines
- Computer vision

Within machine learning, there are different types of learning approaches, such as supervised learning, where a function is inferred from labeled training data. In supervised learning, the system learns to produce correct results based on known outcomes from the training data.

Additionally, deep learning is a specialized area of machine learning that primarily utilizes neural networks. Deep learning involves multiple layers of processing, allowing the model to learn increasingly abstract representations of the data. This approach is particularly effective for complex tasks like speech recognition, image understanding, and handwriting recognition.

Overall, machine learning enables computers to improve their performance on tasks through experience, making it a powerful tool in the field of artificial intelligence.

In [25]:
query = "What is a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A CNN, or Convolutional Neural Network, is a type of deep learning model primarily used for processing structured grid data, such as images. CNNs are designed to automatically and adaptively learn spatial hierarchies of features from input images. They utilize convolutional layers that apply filters to the input data, allowing the model to capture local patterns and features effectively.

In the context of the provided information, CNNs have been a standard approach in image recognition tasks. They have been shown to perform well, especially when trained on large datasets like ImageNet-21k and JFT-300M. The performance of CNNs can scale with the size of the dataset, meaning that as more data becomes available, CNNs can achieve better accuracy in classification tasks.

Recent advancements in image recognition have also seen the emergence of models like the Vision Transformer (ViT), which apply self-attention mechanisms instead of traditional convolutional layers. These models have demonstrated competitive performance with CNNs, particularly when trained on large datasets.

Overall, CNNs remain a foundational technology in the field of computer vision, but the landscape is evolving with the introduction of new architectures that challenge their dominance.

In [26]:
query = "How is a resnet better than a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The provided context does not contain specific information comparing ResNet to CNNs in terms of advantages or improvements. It primarily discusses the Vision Transformer (ViT) and its architecture, performance, and fine-tuning processes, without directly addressing how ResNet is better than CNNs. Therefore, I don't know how a ResNet is better than a CNN based on the given context.

In [27]:
query = "How does a resnet work?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

In [28]:
query = "What is LangGraph?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

In [29]:
query = "What is an Agentic AI System?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The context provided does not contain specific information about an "Agentic AI System." Therefore, I don't know what an Agentic AI System is.

In [30]:
query = "What is LangChain?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.